In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew

In [ ]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

# crewAI Tools

In [ ]:
from crewai_tools import (
  ScrapeWebsiteTool,
  SerperDevTool,
  PDFSearchTool,
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
nurse_dirctory = PDFSearchTool(directory='sources/nurse/EssenTrinken_im_Alter.pdf')

# Creating Agents

In [ ]:
# Agent 1: Pflegefachkraft
nurse = Agent(
    role="geriatric nurse",
    goal="Erstellt eine individuelle Pflegeplanung"
         "für das Ernährungsmanagement bei der Aufnahme {anamnese}"
         "eines neuen Bewohner in der Pflegeeinrichtung,"
         "als Ergbnis der interdisziplnären Zusammenarbeit"
         "mit den anderen Berufsgruppen (Agenten)",
    tools = [scrape_tool, search_tool, nurse_dirctory],
    verbose=True,
    backstory=(
        "Als Pflegefachkraft erstellst du anhand der Anamnese "
        "des Bewohners eine individuelle Pflegeplanung."
        "Du sammelst alle Ergebnisse und Informationen,"
        "von dem Bewohner, dem Arzt und der Diätassistentin. "
        "Du arbeitest Interdisziplinär und du bist zuständig für"
        "den kompletten Pflegeprozess ('https://de.wikipedia.org/wiki/Pflegeplanung') "
        "zum Ernährungsmanagement des Bewohners."
    )
)

In [ ]:
# Agent 2: Ernährungsmediziner
doctor = Agent(
    role="Ernährungsmediziner",
    goal="Abklärung des individuellen Ernährungsrisiko "
         "des Bewohners anhand der medizinischen Angaben {medical}",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "Du gibst eine Einschätzung des Bewohners bezogen auf "
        "sein Ernährungsrisiko aus medizinischer Sicht. "
        "Du analysiert die akuten und chronischen Krankheiten"
        "und verschaffst dir einen Überblick zu den Medikamenten."
        "Die Grundlage für deine Einschätzung sind die allgemein"
        "anerkannten medizinsichen Leitlinien."
    )
)

In [ ]:
# Agent 2: Diätassistentin
diaet = Agent(
    role="Diätassistentin",
    goal="Erstellen eines individuellen Ernährungsplan "
         "anhand der Vorlieben des Bewohners {biografie}.",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "Du bist eine Diätassitentin, spezialsiert auf Ernährungsprobleme"
        "im Alter. Du sammelst alle notwendigen Informationen"
        "damit du für die Küche des Pflegeheimes einen individuellen"
        "Ernährungsplan erstellen kannst. Die "
    )
)

# Creating Tasks

In [ ]:
# Erstellen der Pflegediagnose
nurse_diagnose = Task(
    description=(
        "Erstellen anhand der {anamnese} eine individuelle Risikoeinschätzung "
        "anhand von PEMU. Die Grundlage deiner Einsätzung ist "
        "Expertenstandard Ernährungsmanagement in der Pflege. Deine "
        "Aufgabe ist es bei Einzug des Bewohners alle wichtigen Maßnahmen"
        "zu erfassen, um ein Ernährungsrisiko zu erkennen, falls eines vorhanden ist."
    ),
    expected_output=(
        "Eine systematische pflegerische Einschätzung des"
        "Risiko für Nahrungsmangel und Risiko für Flüssigkeitsmangel"
    ),
    agent=nurse,
    tools = [nurse_dirctory],
    async_execution=True
)

In [ ]:
# Ernährungsmedizinische Einschätzung
medical_diagnose = Task(
    description=(
        "Auf Grundlage der medizinischen Leitlinien analysierst du die physische Situation des Bewohners anhand der vorliegenden Angaben {medical}. "
        "Du gibst Empfehlungen ab, für notwendige medizinische Maßnahmen. "
        "Du analyisert die Medikamente auf ihr Risiko, bezogen zu Ernärhungsproblemen."
        "Du erstellst einen Plan für die Pflegefachkraft, auf welche "
        "Risiken, wie zum Beispiel Wassereinlagerung, Nebenwirkungen, Symptome und"
        "anderes die Pflegefachkraft achten muss."
    ),
    expected_output=(
        "Eine Zusammenfassung der medizinischen Analyse nach: "
        "Mediznische Einschrätzung des Ernährungsrisiko:"
        "Medikamentenempfehlung: "
        "Akute notwendige Untersuchungen: "
        "Symptomkontrolle, Empfehlung pflegerischer Maßnahmen. "
    ),
    agent=doctor,
    output_file="medical_report.md",
    async_execution=True
)

In [ ]:
# Ernärhungsplan
nutrtion_plan = Task(
    description=(
        "Auf Grundlage der ernährungswissenschaftlichen Erkenntnissen und"
        "der Vorlieben der Bewohner, erstellst du einen individuellen Ernährungsplan."
        "Dabei berücksichtigst du, die medizinische Analyse und die Erkenntnisse der Pflegefachraft."
        "Du definierst die notwendigen Kalorien bezogen auf das Ernährungsrisiko und gibts eine Empfehlung für die notwenidige Trinkmenge. "
    ),
    expected_output=(
        "Du erstellst einen indivudllen Ernährungsplan einen Tag in einer Tabelle (csv)"
        "In dieser Tabelle unterteilst du nach folgende Spalten <Frühstück>, <Zwischenmahlzeit>,"
        "<Mittagessen>, <Zwischenmahlzeit>, <Abendessen>, <Gesamtkalorien für einen Tag>."
        "Die Zeilen unterteilst du in die <Wochentage>."
    ),
    agent=diaet,
    output_file="nutrtion_plan.csv",
    context=[medical_diagnose],
    async_execution=False
)

In [ ]:
# Erstellen der Pflegeplanung
care_process = Task(
    description=(
        "Auf Grundlage der pflegerischen, medizinischen und ernährungswissenschaftlichen Erkenntnissen, analysierst du das Ernährungsrisiko und erstellst eine individuelle Pflegeplanung für den Bewohner."
    ),
    expected_output=(
        "Du erstellst am Ende aller Erkenntnisse eine individuelle Pflegeplanung zum Ernährungsmanagement für den Bewohner."
        "Du beschreibst das Pflegeproblem und die Resourcen des Bewohners."
        "Du definierst die Pflegediagnose."
        "Du definierst ein Pflegeziel."
        "Du beschreibst die geeigneten Maßnahmen."
        "Du definierst wann der Pflegeprozess geprüft werden soll."
    ),
    agent=nurse,
    output_file="care_plan.md",
    async_execution=False
)

# Creating the Crew

In [ ]:
# Define the crew with agents and tasks
nutrtion_risk_crew = Crew(
    agents=[nurse, 
            doctor, 
            diaet],
    
    tasks=[nurse_diagnose, 
           medical_diagnose, 
           nutrtion_plan,
           care_process],
    
    verbose=True
)

# Der Einzug erfolgt, der Pflegeplan wird erstellt

In [ ]:
anmanese_details_input = {
    'anamnese': "Herr Karl, Maier ist 85 Jahre alt. Er ist 175cm groß und er wiegt 65kg. Er hat in den letzten 6 Monaten 7kg abgenommen. Er hat bisher alleine zuhause gelebt, aber ist hier nicht mehr zurechtgekommen. Seine Frau ist vor 8 Monaten verstorben und er konnte sich nicht mehr versorgen. Er hat eine mittelgradige Demenz und er konnte seinen Einkauf nicht mehr organisieren. zuletzt musste er wegen Exikkose in die Klinik, weil er zu wenig getrunken hat. Er ist eigentlich noch sehr mobil und kann selbständig gehen, allerdings ist er vor 3 Monaten das erste mal gestürzt, jedoch ohne Folgen. Er hat noch eigene Zähne, er fühlt sich aber in der letzten Zeit immer sehr müde. Zur Körperpflege reicht es wenn man ihn auffordert. Überschluckstörungen klagt er nicht. Er kann auch noch feste Speisen schneiden und alleine essen. Er klagt auch darüber, dass er immer schlechter sehen kann. Er hat aber nur eine Tochter, die in Amerika lebt. Arztbesuche konnte er nicht mehr für sich organisieren, wegen seiner Demenz. Er hat einen Betreuer. ",
    'medical': "Herr Karl, Maier hat folgende Diagnosen: Mittelgradige Demenz, Herzinsuffizienz, COPD GOLD Stadium 2, Coxarthrose links. Vor drei Monaten war er im Krankenhaus wegen einer Exikkose. Seite Vitalzeichen sind unauffällig, er hat jedoch auffällig an Gewicht abgenommen, seit dem Tod seiner Frau. Deswegen bekommt er Citalopram und wenn er am Abend nicht schlafen kann, kann er eine Tavor nehmen. ER nimmt noch folgende Medikamente, am Morgem Ramipril 5mg, Metoprolol 47,5mg und Simvastatin.",
    'biografie': "Herr Karl, Maier ist am liebsten deftig. Er mag kein Obst und Gemüse. Er isst am liebsten Fleisch und Wurst. Allergien und Unverträglichkeiten sind nicht bekannt. In letzter Zeit hat sich aber meistens nur von Süßigkeiten ernährt, weil ihm nichts mehr wirklich schmeckt.",
}

In [ ]:
### this execution will take a few minutes to run
result = nutrtion_risk_crew.kickoff(inputs=anmanese_details_input)

In [ ]:
from IPython.display import Markdown, display
display(Markdown("care_plan.md"))

In [ ]:
import pandas as pd
from tabulate import tabulate
from IPython.display import Markdown

# CSV-Datei laden
df = pd.read_csv("nutrtion_plan.csv")

# DataFrame als Markdown-Tabelle anzeigen
markdown_table = tabulate(df, headers='keys', tablefmt='pipe', showindex=False)
display(Markdown(markdown_table))
